# Payday Loan Vendor Capture

In [17]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
g_key = os.environ.get('GKEY')

In [3]:
# Load the csv containing the zip codes into a dataframe
counties_zips_df = pd.read_csv("resources/CACountiesZips.csv", encoding="utf-8")
counties_zips_df

,Zip Code,City,County
0,90001,Los Angeles,Los Angeles
1,90002,Los Angeles,Los Angeles
2,90003,Los Angeles,Los Angeles
3,90004,Los Angeles,Los Angeles
4,90005,Los Angeles,Los Angeles
...,...,...,...
2652,96157,South Lake Tahoe,El Dorado
2653,96158,South Lake Tahoe,El Dorado
2654,96160,Truckee,Nevada
2655,96161,Truckee,Nevada


In [5]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [36]:
# Construct an empty dataframe to hold search results
payday_df = pd.DataFrame(columns=['Zip Code','Payday Vendor','Address','Lat','Lng'])

payday_df

,Zip Code,Payday Vendor,Address,Lat,Lng


In [57]:

#for index, row in hotel_df.iterrows():

# Create a variable to hold the zip code
zip = "93710"

# Params dictionary to update each iteration
params = {
    "query": f"payday+loan+in+{zip}",
    "radius": 0,
    "key": g_key
}

# Create a variable to hold the base url for the search
base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"

# Make request and store results
payday_list = requests.get(base_url, params=params)
    
# Convert to json
payday_list = payday_list.json()

#Create a variable to hold the index
index=0

# Create a for loop to write correct results to the dataframe filter out result not containing the searched zip code
for x in payday_list["results"]:
    # Try-except to skip any that are missing data.
    try:
        test = x['formatted_address']
        if zip in test:
            print("Payday loan vendor found... writing to dataframe")
            payday_df.loc[index, 'Zip Code'] = zip
            payday_df.loc[index, 'Payday Vendor'] = x['name']
            payday_df.loc[index, 'Address'] = x['formatted_address']
            payday_df.loc[index, 'Lat'] = x['geometry']['location']['lat']
            payday_df.loc[index, 'Lng'] = x['geometry']['location']['lng']
            index += 1
        else:
            print("Skipping location \ not in zipcode")
            pass

    except (KeyError, IndexError): 
        print("Missing field/result... skipping.")
        
payday_df.head()

Payday loan vendor found... writing to dataframe
Payday loan vendor found... writing to dataframe
Payday loan vendor found... writing to dataframe
Payday loan vendor found... writing to dataframe
Payday loan vendor found... writing to dataframe
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Payday loan vendor found... writing to dataframe
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode
Skipping location \ not in zipcode


,Zip Code,Payday Vendor,Address,Lat,Lng
0,93710,Advance America,"5054 N Blackstone Ave Ste. 102, Fresno, CA 937...",36.8096,-119.79
1,93710,California Budget Finance,"785 E Shaw Ave, Fresno, CA 93710, United States",36.8081,-119.773
2,93710,ACE Cash Express,"432 E Bullard Ave, Fresno, CA 93710, United St...",36.8234,-119.781
3,93710,ACE Cash Express,"785 E Shaw Ave, Fresno, CA 93710, United States",36.8081,-119.773
4,93710,Cash Plus,"1057 E Shaw Ave, Fresno, CA 93710, United States",36.8073,-119.771


In [61]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Payday Vendor}</dd>
<dt>Zip Code</dt><dd>{Zip Code}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
vendor_info = [info_box_template.format(**row) for index, row in payday_df.iterrows()]
locations = payday_df[["Lat", "Lng"]]

In [62]:
# Add marker layer
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=vendor_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))